In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyRegressor
import math

In [2]:
df = pd.read_csv("../dataset/dataset.csv")

In [3]:
X = df[["plain_text_len", "question_marks", "esclamation_marks", "emojis", "hashtags", "tags", "urls", "verbs", "nouns", "adjs", "advs"]].to_numpy()
y = df[["retweet_count"]].to_numpy()

In [4]:
#from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
from sklearn.model_selection import KFold

kfold_grid = KFold(n_splits=3, shuffle=True, random_state=7)
kfold_score = KFold(n_splits=3, shuffle=True, random_state=3)
metrics = ("neg_root_mean_squared_error", "sd")

# Trivial Predictor
Calcola la media

In [5]:
model = "TP"
dr = DummyRegressor(strategy="mean")
sd_scores = cross_validate(dr, X, y.ravel(), scoring=metrics[0], return_train_score=True, cv=kfold_score, n_jobs=-1)

#sd_scores1 = []
#
# VERIFICA
#for train_index, test_index in kfold_score.split(X):
#    X_train, X_test = X[train_index], X[test_index]
#    y_train, y_test = y[train_index], y[test_index]
#    dr.fit(X_train, y_train.ravel()) # calcola media
#    dr_predictions = dr.predict(X_test) # predice tutti i valori
#    fold_sd_score = np.power(np.power(y_test.ravel()-dr_predictions, 2).mean(), 1/2)
#    sd_scores1.append(fold_sd_score)
#
#print("{} {} cross-val test scores (manually): ".format(model, metrics[1]), sd_scores1)

# SD
print("{} {} cross-val test scores: ".format(model, metrics[1]), sd_scores["test_score"])

print("{} {} test mean: {:.2f}".format(model, metrics[1], sd_scores["test_score"].mean()))
print("{} {} test std: {:.2f}".format(model, metrics[1], sd_scores["test_score"].std()))

print("{} {} train mean: {:.2f}".format(model, metrics[1], sd_scores["train_score"].mean()))
print("{} {} train std: {:.2f}".format(model, metrics[1], sd_scores["train_score"].std()))

TP sd cross-val test scores:  [ -9.85331661  -9.56552883 -10.57559733]
TP sd test mean: -10.00
TP sd test std: 0.42
TP sd train mean: -10.00
TP sd train std: 0.22


# k-Nearest Neighbors
(trivial predictor #2)
Training is performed with Grid Search and Cross-Validation

In [6]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

parameters = {
    'kneighborsregressor__n_neighbors': [5, 10, 15]
}

clf = KNeighborsRegressor()
pipe = make_pipeline(StandardScaler(), clf)

best_model = GridSearchCV(pipe, param_grid=parameters, cv=kfold_grid)
# best_model.fit(X_train, y_train.ravel())

In [7]:
# print("KNR best parameters: []".format(best_model.best_params_))
# print("KNR best cross-val score: {:.2f}".format(best_model.best_score_))

model = "KNN"
scores = cross_validate(best_model, X, y.ravel(), cv=kfold_score, scoring=metrics[0], return_train_score=True, n_jobs=-1)

# RMSE
print("{} {} cross-val test scores: ".format(model, metrics[0]), scores["test_score"])

print("{} {} test mean: {:.2f}".format(model, metrics[0], scores["test_score"].mean()))
print("{} {} test std: {:.2f}".format(model, metrics[0], scores["test_score"].std()))

print("{} {} train mean: {:.2f}".format(model, metrics[0], scores["train_score"].mean()))
print("{} {} train std: {:.2f}".format(model, metrics[0], scores["train_score"].std()))

KNN neg_root_mean_squared_error cross-val test scores:  [-7.31761723 -7.07534947 -7.57913355]
KNN neg_root_mean_squared_error test mean: -7.32
KNN neg_root_mean_squared_error test std: 0.21
KNN neg_root_mean_squared_error train mean: -6.74
KNN neg_root_mean_squared_error train std: 0.16


# Random forest

Training is performed with Grid Search and Cross-Validation

In [8]:
from sklearn.ensemble import RandomForestRegressor

parameters = {
    'max_depth': [3, 5, 10],
    'n_estimators': [100, 500, 1000]
}

rfr = RandomForestRegressor(n_jobs=-1)

best_model = GridSearchCV(rfr, param_grid=parameters, cv=kfold_grid)
# best_model.fit(X_train, y_train.ravel())

In [9]:
# print("RF best parameters: []".format(best_model.best_params_))
# print("RF best cross-val score: {:.2f}".format(best_model.best_score_))
# print("RF test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

model = "RF"
scores = cross_validate(best_model, X, y.ravel(), cv=kfold_score, scoring=metrics[0], return_train_score=True, n_jobs=-1)

# RMSE
print("{} {} cross-val test scores: ".format(model, metrics[0]), scores["test_score"])

print("{} {} test mean: {:.2f}".format(model, metrics[0], scores["test_score"].mean()))
print("{} {} test std: {:.2f}".format(model, metrics[0], scores["test_score"].std()))

print("{} {} train mean: {:.2f}".format(model, metrics[0], scores["train_score"].mean()))
print("{} {} train std: {:.2f}".format(model, metrics[0], scores["train_score"].std()))

RF neg_root_mean_squared_error cross-val test scores:  [-7.02812343 -6.72792903 -7.47683205]
RF neg_root_mean_squared_error test mean: -7.08
RF neg_root_mean_squared_error test std: 0.31
RF neg_root_mean_squared_error train mean: -6.36
RF neg_root_mean_squared_error train std: 0.31


# Support Vector Machine

Training is performed with Standard scaling followed by Grid Search and Cross-Validation

In [10]:
from sklearn.svm import SVR

parameters = {
    'svr__kernel': ["linear", "poly", "rbf"],
    'svr__C': [0.01, 0.1, 1, 10, 100, 1000] # valore alto = maggior importanza alle singole osservazioni
}

pipe = make_pipeline(StandardScaler(), SVR())

best_model = GridSearchCV(pipe, param_grid=parameters, cv=kfold_grid)
# best_model.fit(X_train, y_train.ravel())

In [11]:
# print("SVR best parameters: []".format(best_model.best_params_))
# print("SVR best cross-val score: {:.2f}".format(best_model.best_score_))

Model = "SVR"
scores = cross_validate(best_model, X, y.ravel(), cv=kfold_score, scoring=metrics[0], return_train_score=True, n_jobs=-1)

# RMSE
print("{} {} cross-val test scores: ".format(model, metrics[0]), scores["test_score"])

print("{} {} test mean: {:.2f}".format(model, metrics[0], scores["test_score"].mean()))
print("{} {} test std: {:.2f}".format(model, metrics[0], scores["test_score"].std()))

print("{} {} train mean: {:.2f}".format(model, metrics[0], scores["train_score"].mean()))
print("{} {} train std: {:.2f}".format(model, metrics[0], scores["train_score"].std()))

RF neg_root_mean_squared_error cross-val test scores:  [-7.20828344 -7.25046513 -7.87478343]
RF neg_root_mean_squared_error test mean: -7.44
RF neg_root_mean_squared_error test std: 0.30
RF neg_root_mean_squared_error train mean: -6.66
RF neg_root_mean_squared_error train std: 0.09
